```{currentmodule} optimap
```

In [ ]:
# Code snippet for rendering animations in the docs
from IPython.display import HTML
import warnings
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128

def render_ani_func(f):
    om.utils.disable_interactive_backend_switching()
    plt.switch_backend('Agg')
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        ani = f()
    %matplotlib inline
    om.utils.enable_interactive_backend_switching()

    vid = HTML(ani.to_html5_video())
    plt.close('all')
    return vid

```{tip}
Download this tutorial as a {download}`Jupyter notebook <converted/basics.ipynb>`, or a {download}`python script <converted/basics.py>` with code cells.
```

# Tutorial 1: Basics

This tutorial will walk you through the basics of using the `optimap` package.

First, let's import optimap and the other packages we will need:


In [ ]:
import optimap as om
import numpy as np
import matplotlib.pyplot as plt

## Loading a video file

We now have access to all the functions in the `optimap` package. Use the function {func}`optimap.load_video` to load a video file. The following file formats are supported:

* .tif, .tiff (TIFF stack)
* .dat (MultiRecorder)
* .gsd, .gsh (SciMedia MiCAM 05)
* .rsh, .rsm, .rsd (SciMedia MiCAM ULTIMA)
* .npy (numpy array)
* .mat (MATLAB), loads the first field in the file

Here, we first download a sample recording of a beating Rabbit heart stained with the voltage-sensitive dye di-4-ANEPPS was acquired at 500 fps using a Basler acA720-520um camera. The action potentials are inverted, i.e. an upstroke is observed as a negative deflection. We have extracted a short section of the original recording and saved the raw data as a numpy file (`.npy`). Experimenters: Jan Lebert, Shrey Chowdhary & Jan Christoph (University of California, San Francisco, USA).

In [ ]:
filepath = om.utils.retrieve_example_data('Example_02_VF_Rabbit_Di-4-ANEPPS_Basler_acA720-520um.npy')
video = om.load_video(filepath)

om.print_properties(video)

optimap imports videos as numpy arrays with the shape (Time, Height, Width). This convention is used throughout the library. See {func}`load_video` for additional arguments, e.g. to load only a subset of the frames or to use memory mapping to reduce memory usage.

```python
video = om.load_video('Example.dat', start_frame=100, frames=1000, step=2, use_mmap=True)
```

For some file formats, optimap can also load the corresponding metadata using {func}`load_metadata`. For example, the following code loads the metadata of a MiCAM ULTIMA recording:

```python
metadata = om.load_metadata('Example.rsh')
```

To crop, rotate, or flip a video see {mod}`optimap.video` for a list of available functions.

## Playing videos
Videos can be viewed using either:
1. the built-in viewer {func}`play_video` based on matplotlib
2. using Monochrome, which is a more advanced viewer and allows for more interactivity

### Using the built-in viewer

In [ ]:
om.play_video(video);

In [ ]:
render_ani_func(lambda: om.video.play(video))

See API documentation for {func}`play_video` for a list of available arguments, e.g.:
```python
om.play_video(video, title='Example video', vmin=0, vmax=1, cmap='gray', interval=20)
```


### Using Monochrome

In [ ]:
import monochrome as mc
mc.show(video, "raw video")

See the {doc}`monochrome` documentation and {func}`monochrome.show` for more information. For example, click in the video to view time traces at the selected positions.

## Viewing and extracting traces

Time traces can be viewed and extracted interactively using the {func}`select_traces` function. Click on the image to select positions, right click to remove positions. Close the window to continue.

In [ ]:
traces, positions = om.select_traces(video, size=5)

In [ ]:
positions = [(127, 147), (130, 209), (202, 136)]

fig, axs = plt.subplots(1,2, figsize=(11,4))
om.trace.show_positions(video[0], positions, ax=axs[0])
traces = om.trace.extract_traces(video, positions, size=5, ax=axs[1], show=True, fps=500)
plt.show()

Traces are averaged spatially over a small window surrounding the chosen location, the `size` parameter controls the dimensions of the window. By default, this window is a rectangle with dimensions `(size, size)`. In this case (the default), the window is 5 by 5 pixels.

To get the exact pixel values without spatial averaging, set `size=1`.

If you'd like to display the time axis in seconds rather than frames, use the `fps` (frames per second) parameter.

In [ ]:
traces = om.extract_traces(video, positions, size=1, show=True, fps=500)

The `window` parameter can be used to define the window function, `'disc'` uses a circular region with diameter `size` around the position. See {func}`select_traces` for more information and {func}`optimap.trace.set_default_trace_window` to change the default window type.

Internally {func}`extract_traces` uses {func}`show_traces` to plot traces. In general, all plotting functions in optimap have an `ax` parameter which can be used to specify a custom matplotlib axes object.

For example, we can create a figure with two subplots and show the positions on the first subplot and the traces on the second subplot with milliseconds as time unit:

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(10,5))

om.trace.show_positions(video[0], positions, ax=axs[0])

x_axis_ms = (np.arange(video.shape[0]) / 500.0) * 1000
traces = om.extract_traces(video[:300],
                           positions,
                           x=x_axis_ms[:300],
                           size=5,
                           window='disc',
                           ax=axs[1],
                           show=True)
axs[1].set_xlabel('Time [ms]')
plt.show()

## Motion Compensation

The heart is beating and slightly moving during this recording. Even though the motion is small, it can have a strong effect on the time traces in the form of motion artifacts and prevent further analysis. We can use the {func}`motion_compensate` function to compensate for the motion using the steps described in {cite}`Christoph2018a` and {cite}`Lebert2022`. See [](motion_compensation) for detailed information and examples.

In [ ]:
warped = om.motion_compensate(video,
                              contrast_kernel=5,
                              presmooth_spatial=1,
                              presmooth_temporal=1)

Let's view the original video and motion-compensated video side by side using {func}`optimap.video.play2`:

In [ ]:
om.video.play2(video,
               warped,
               title1="with motion",
               title2="without motion",
               skip_frame=3);

In [ ]:
render_ani_func(lambda: om.video.play2(video, warped, title1="with motion", title2="without motion", skip_frame=1))

## Saving and rendering videos

Let's save the motion-compensated recording as a tiff stack and also render it to a .mp4 video file.

In [ ]:
om.video.save_video(warped, 'warped_recording.tiff')
om.video.export_video(warped, 'warped_recording.mp4', fps=50)

The optimap video-player functions such as {func}`optimap.video.play2` can also be exported to a video file:
```python
animation = om.video.play2(video, warped, title1='Raw', title2='Compensated')
animation.save('Example.mp4')
```
See {meth}`matplotlib.animation.Animation.save` for more details.

## Fluorescence wave isolation

To better visualize the action potential propagation we can compute a pixel-wise normalization to [0, 1] using a sliding/rolling window of 60 frames.

In [ ]:
norm_raw = om.video.normalize_pixelwise_slidingwindow(video, window_size=60)
norm_warped = om.video.normalize_pixelwise_slidingwindow(warped, window_size=60)
om.video.play2(norm_raw, norm_warped, title1="with motion", title2="without motion");

In [ ]:
def f():
    norm_raw = om.video.normalize_pixelwise_slidingwindow(video, window_size=60)
    norm_warped = om.video.normalize_pixelwise_slidingwindow(warped, window_size=60)
    return om.video.play2(norm_raw[:500], norm_warped[:500], title1="with motion", title2="without motion", interval=20)
render_ani_func(f)

In [ ]:
mask = om.background_mask(warped[0])
# norm_warped[:, mask] = 0